Import libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
import sklearn.metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold, cross_validate, permutation_test_score
import statistics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

Import data

In [ ]:
# feature data
file = None #path to file

# video metadata
file_metadeta = None #path to file

In [3]:
df_stats = pd.read_csv(file,header=0)
print(df_stats.shape)                 

(759, 381)


get metadata and merge

In [4]:
# read in metadata
df_metadata = pd.read_csv(file_metadata,header=0, sep=';')
df_metadata.head()

# get data for merging
df_metadata_merge = df_metadata[['videoname', 'Trial_time','Genotype', 'Genotype_class','Mouseline']] 

# merged data with labels and trial phase
df_data_total = df_stats.merge(df_metadata_merge, on = 'videoname', how = 'left')

Change string variable to numeric

In [5]:
dict_test_phases = {'Test': 0, 'Sample1': 1, 'Sample2': 2, 'Sample3': 3}
for index,row in df_data_total.iterrows():
    value = row['Trial_time']
    df_data_total.at[index,'Trial_time'] = dict_test_phases[value]

define different label conditions

In [6]:
# TG vs WT
y_2 = df_data_total['Genotype_class'].copy()


# for multiclass classification

# 8 TG classes vs WT
y_9 = df_data_total['Genotype'].copy()
for i in y_9:
    if 'WT' in i:
        y_9 = y_9.replace(i,'WT')

# 8 TG classes and 7 WT classes
y_15 = df_data_total['Genotype'].copy()

# for changing labels to numeric incase function later complains for classificiation
# add labels + number in dict
#dict_y2 = 
#dict_y9 = 
#dict_y15 = 
#for i in range(len(y_2)):
#    y_2[i] = dict_y2[y_2[i]]
 #   y_9[i] = dict_y9[y_9[i]]
 #   y_15[i] = dict_y15[y_15[i]]

Create headers

In [7]:
# total header
header_total = list(df_data_total.columns.values)

columns_to_remove = ['Genotype_class', 'videoname', 'Genotype', 'Mouseline']
for column in columns_to_remove:
    header_total.remove(column)
    
# header mean columns for baseline model
header_mean = [i for i in header_total if 'mean' in i]
header_mean.append('Trial_time')

# create new dataset without mean features
header_total_minmean = []
for column in header_total:
    if 'mean' not in column:
        header_total_minmean.append(column)

# header without median and mean columns
header_without_median = [i for i in header_total_minmean if 'median' not in i]

# header without var and mean columns
header_without_var = [i for i in header_total_minmean if 'var' not in i]

# header without std and mean columns
header_without_std = [i for i in header_total_minmean if 'std' not in i]

# header only SD
header_sd = [i for i in header_total_minmean if 'std' in i]
header_sd.append('Trial_time')

# header only variance
header_var = [i for i in header_total_minmean if 'var' in i]
header_var.append('Trial_time')

# header only median
header_median = [i for i in header_total_minmean if 'median' in i]
header_median.append('Trial_time')

Create datasets

In [10]:
# Baseline data with mean
x_mean = df_data_total[header_mean]

# large dataset without taking columns out
x_whole = df_data_total[header_total]

# extended model median-var-sd
x_extended = df_data_total[header_total_minmean]

# data without median
x_without_median = df_data_total[header_without_median]

# data without var
x_without_var = df_data_total[header_without_var]

# data without std
x_without_std = df_data_total[header_without_std]

# data only SD
x_sd = df_data_total[header_sd]

# data only variance
x_var = df_data_total[header_var]

# data only median
x_median = df_data_total[header_median]

Run pipeline with scaler, pca and rf

In [14]:
# for easy changing
# baseline = x_mean
# extended model median-var-sd = x_extended

# used data -> change titles and exported figures names
data = x_extended
label = y_2

# define lists for scores
score_accuracy = []
score_f1 = []
score_precision = []
score_recall = []
support = []

# other variables
count = 0
confusion_total = []
PCA_comp = []
small_len_pca = None

# define kfold
kf =KFold(n_splits=5, shuffle = True, random_state = 42)

for train_index, test_index in kf.split(data):
    x_train, x_test = data.loc[train_index], data.loc[test_index]
    y_train, y_test = label[train_index], label[test_index]
    count += 1
    
    #define pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()), # can be adjusted
        ('pca', PCA(0.95,random_state = 42)),  # now includes num components needed for 95% variance, can be adjusted
        ('rf', RandomForestClassifier(random_state = 42))])  # tune parameters if needed

    # fit data and predict
    pipeline.fit(x_train, y_train)
    model_predicts = pipeline.predict(x_test)
    scores_report = classification_report(y_test, model_predicts, output_dict=True) 
    
    # get std and mean of scores
    score_accuracy.append(scores_report['accuracy'])
    support.append((scores_report['TG']['support'],scores_report['WT']['support']))
    score_f1.append(scores_report['macro avg']['f1-score'])
    score_precision.append(scores_report['macro avg']['precision'])
    score_recall.append(scores_report['macro avg']['recall'])
    
    # plot explained variance for each fold + append for overarching figure
    explained_variance = pipeline.steps[1][1].explained_variance_ratio_
    PCA_comp.append(explained_variance)
    print(len(explained_variance))
    
    # get smallest component for cross-validation figure
    if small_len_pca == None:
        small_len_pca = len(explained_variance)
    else:
        if len(explained_variance) < small_len_pca:
            small_len_pca = len(explained_variance)
            
    cumulative_variance= np.cumsum(explained_variance)
    plt.bar(range(0,len( explained_variance)), explained_variance, alpha=0.5, align='center', label='Individual explained variance')
    plt.step(range(0,len(cumulative_variance)), cumulative_variance, where='mid',label='Cumulative explained variance')
    plt.title('Explained variance over components\n extended model, fold %i ' %count)
    plt.ylabel('Ratio explained variance')
    plt.xlabel('Principal component index')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig('/extended_variance%i.png'%count) #add output path
    plt.clf()

    # save confusion matrix values for overarching results
    confusion = confusion_matrix(y_test,model_predicts, labels = ['TG','WT'])
    confusion_total.append(confusion)
    
# print scores  
print("Accuracy mean:",  sum(score_accuracy) / len(score_accuracy),
      "Accuracy std:", np.std(score_accuracy),
      "Precision mean", sum(score_precision) / len(score_precision),
      "Precision std:", np.std(score_precision),
      "Recall mean:", sum(score_recall) / len(score_recall),
      "Recall std:", np.std(score_recall),
      "F1 mean:", sum(score_f1) / len(score_f1),
      "F1 std:", np.std(score_f1),
      "Support:", support)
 
# permutation test
score_pipeline, permutation_scores, pvalue = permutation_test_score(pipeline, data, label, scoring="accuracy", cv=kf, n_permutations=200)
print('pvalue permutation')
print(pvalue)
print('score permutation')
print(score_pipeline)

# plot permutation scores + value test 
fig, ax = plt.subplots()
ax.hist(permutation_scores, bins=20, density=True)
ax.axvline(score_pipeline, ls='--', color='r', label=' accuracy original')
ax.set_xlabel("Accuracy")
_ = ax.set_ylabel("Test amount")
plt.legend(loc='best')
plt.title('Permutation test results, extended model')
plt.savefig('/extended_permutation_withtext.png') #add output path
plt.clf()

# show confusion matrix as heatmap
result_confusion = sum(confusion_total) 
disp = ConfusionMatrixDisplay(confusion_matrix=result_confusion,display_labels=['TG','WT'])
disp.plot()
plt.title('Confusion matrix, extended model')
plt.savefig('/extended_confusion.png') #add output path
plt.clf()

# get smallest PCA components over values
new_PCA_comp = []
for fold in PCA_comp:
    new_PCA_comp.append(fold[:small_len_pca])
components_averaged = sum(new_PCA_comp)/5
print('comp for every fold')
print(PCA_comp)
print('averaged')
print(components_averaged)

# save overarching component figure
cumulative_variance_total = np.cumsum(components_averaged)
plt.bar(range(0,len(components_averaged)),components_averaged, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cumulative_variance_total)), cumulative_variance_total, where='mid',label='Cumulative explained variance')
plt.title('Explained variance summed over cross-validation\n extended model')
plt.ylabel('Ratio explained variance')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('/extended_variance.png') # add output path
plt.clf()

27
27
27
29
27
Accuracy mean: 0.6257668177065179 Accuracy std: 0.030873811474637688 Precision mean 0.6230258778691582 Precision std: 0.02988243972164329 Recall mean: 0.6139413509817008 Recall std: 0.026707806891417826 F1 mean: 0.6122825331230612 F1 std: 0.027941604892639874 Support: [(85, 67), (78, 74), (88, 64), (80, 72), (81, 70)]
pvalue permutation
0.004975124378109453
score permutation
0.6257668177065179
comp for every fold
[array([0.36880685, 0.10063221, 0.08368255, 0.07736951, 0.05285658,
       0.04681132, 0.03227904, 0.02594482, 0.02051913, 0.01868393,
       0.01428771, 0.01349679, 0.01279772, 0.01077125, 0.00945045,
       0.00849026, 0.00770947, 0.00708744, 0.00608705, 0.00508444,
       0.00479097, 0.00457144, 0.00414668, 0.00395712, 0.00341298,
       0.00323922, 0.00312176]), array([0.40124595, 0.09756278, 0.07878779, 0.07675382, 0.04427099,
       0.03733286, 0.03180625, 0.02686694, 0.02056279, 0.01823047,
       0.01540451, 0.01359921, 0.01223716, 0.01020684, 0.00926429

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

Run random forest alone

In [17]:
# for easy changing
# baseline = x_mean
# extended model median-var-sd = x_extended

# used data -> change titles and exported figures names
data = x_extended
label = y_2

# define lists for scores
score_accuracy = []
score_f1 = []
score_precision = []
score_recall = []
support = []

# other variables
count = 0
confusion_total = []
PCA_comp = []
small_len_pca = None

# define kfold
kf =KFold(n_splits=5, shuffle = True, random_state = 42)

for train_index, test_index in kf.split(data):
    x_train, x_test = data.loc[train_index], data.loc[test_index]
    y_train, y_test = label[train_index], label[test_index]
    count += 1
    
    #define pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()), # can be adjusted
        ('rf', RandomForestClassifier(random_state = 42))]) # can adjust parameters

    # fit data and predict
    pipeline.fit(x_train, y_train)
    model_predicts = pipeline.predict(x_test)
    scores_report = classification_report(y_test, model_predicts, output_dict=True) 
    
    # get std and mean of scores
    score_accuracy.append(scores_report['accuracy'])
    support.append((scores_report['TG']['support'],scores_report['WT']['support']))
    score_f1.append(scores_report['macro avg']['f1-score'])
    score_precision.append(scores_report['macro avg']['precision'])
    score_recall.append(scores_report['macro avg']['recall'])
    
    # Extract the feature importances from the best estimator
    feature_importances = pipeline[1][1].feature_importances_
    
    # Create a DataFrame to hold the feature importances and their corresponding names
    importance_df = pd.DataFrame({'feature': data.columns.values, 'importance': feature_importances})
    
    # Sort the DataFrame by importance
    importance_df = importance_df.sort_values('importance', ascending=False)
    a = importance_df.loc[importance_df['importance'].lt(0.001), 'importance'].idxmax()
    print('feature importances')
    print(importance_df.head(5))
    # Visualize the feature importances as a bar plot
    plt.figure(figsize=(8, 6))
    sns.barplot(data=importance_df.loc[:a], x='importance', y='feature', palette='viridis')
    plt.title('Feature Importances fold %i' %count)
    plt.savefig('/extended_importance_rf,fold%i.png' %count) #adjust path
    plt.clf()

    # Visualize the confusion matrix as a heatmap
    confusion = confusion_matrix(y_test,model_predicts, labels = ['TG','WT'])
    confusion_total.append(confusion)
    
# print scores  
print("Accuracy mean:",  sum(score_accuracy) / len(score_accuracy),
      "Accuracy std:", np.std(score_accuracy),
      "Precision mean", sum(score_precision) / len(score_precision),
      "Precision std:", np.std(score_precision),
      "Recall mean:", sum(score_recall) / len(score_recall),
      "Recall std:", np.std(score_recall),
      "F1 mean:", sum(score_f1) / len(score_f1),
      "F1 std:", np.std(score_f1),
      "Support:", support)
 
# permutation test
score_pipeline, permutation_scores, pvalue = permutation_test_score(pipeline, data, label, scoring="accuracy", cv=kf, n_permutations=200)
print('pvalue permutation')
print(pvalue)
print('score permutation')
print(score_pipeline)

# plot permutation scores + value test 
fig, ax = plt.subplots()
ax.hist(permutation_scores, bins=20, density=True)
ax.axvline(score_pipeline, ls='--', color='r', label=' accuracy original')
ax.set_xlabel("Accuracy")
_ = ax.set_ylabel("Test amount")
plt.legend(loc='best')
plt.title('Permutation test results, extended model')
plt.savefig('/extended_permutation_withtext.png') #adjust output path
plt.clf()

# print confusion matrix
result_confusion = sum(confusion_total) 
disp = ConfusionMatrixDisplay(confusion_matrix=result_confusion,display_labels=['TG','WT'])
disp.plot()
plt.title('Confusion matrix, extended model')
plt.savefig('/extended_confusion.png') #adjust output path
plt.clf()

feature importances
                               feature  importance
73   var_Area_nose_schoulderL_tailbase    0.055410
263       std_Angle_earL_tailbase_earR    0.044445
232     std_Angle_nose_earL_schoulderR    0.035740
230     std_Angle_nose_schoulderL_earL    0.028077
142    median_Angle_earL_earR_tailbase    0.027999
feature importances
                                   feature  importance
94       std_Area_nose_schoulderR_tailbase    0.062594
135      median_Angle_earR_earL_schoulderL    0.051127
87           std_Area_nose_earL_schoulderR    0.042503
93       std_Area_nose_schoulderL_tailbase    0.041360
158  median_Angle_earR_tailbase_schoulderL    0.035239
feature importances
                                     feature  importance
203             var_Angle_earL_tailbase_earR    0.062078
71               var_Area_nose_earR_tailbase    0.053568
231           std_Angle_earL_nose_schoulderR    0.046580
84   var_Area_schoulderL_schoulderR_tailbase    0.046334
211       var_Angle

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>